In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
df_shp = gpd.read_file("../data/geoshape/cb_2018_us_cbsa_20m/cb_2018_us_cbsa_20m.shp")
df_hourly_search = pd.read_csv("../data/google_trend/DerailmentHourly.csv", sep="\t", index_col="DMA")
df_daily_search = pd.read_csv("../data/google_trend/DerailmentDaily.csv", sep="\t", index_col="DMA")

In [24]:
geometry = [Point(xy) for xy in zip(df_hourly_search["Longitude"], df_hourly_search["Latitude"])]
df_geo = gpd.GeoDataFrame(df_hourly_search, geometry=geometry, crs="EPSG:4269")
df_geo = gpd.sjoin(df_geo, df_shp, predicate="within", how="left")

dfs = list()
df_geo["Date"] = pd.to_datetime(df_geo["EST"])
df_geo = df_geo.set_index(["Date"])
for dt in df_geo.index.drop_duplicates():
    df = df_geo.loc[dt]
    gdf = df_shp.set_index(["NAME"])
    gdf["Count"] = df.groupby(["NAME"]).sum()["DMA_Search"]
    gdf["Count"] = gdf["Count"].fillna(0)
    gdf["EST"] = dt
    dfs.append(gdf)
df_geo = pd.concat(dfs).sort_values(["EST"])
df_geo.head(5)

,GEOID,CSAFP,CBSAFP,AFFGEOID,LSAD,ALAND,AWATER,Population,geometry,Count,EST
NAME,,,,,,,,,,,
"Muskogee, OK",34780,538,34780,310M400US34780,M2,2098872174,76147852,66146.0,"POLYGON ((-95.76617 35.85640, -95.65042 35.856...",0.0,2023-02-03 22:00:00
"Greenville, MS",24740,NaN,24740,310M400US24740,M2,1877059426,94506019,43687.0,"POLYGON ((-91.21567 33.52942, -90.76351 33.529...",5.0,2023-02-03 22:00:00
"Columbia, SC",17900,192,17900,310M400US17900,M1,9591002754,339370306,838250.0,"POLYGON ((-81.99071 33.99661, -81.87008 34.135...",3.0,2023-02-03 22:00:00
"Twin Falls, ID",46300,NaN,46300,310M400US46300,M1,6524657692,31157724,116905.0,"POLYGON ((-115.03770 42.76841, -115.03777 42.9...",0.0,2023-02-03 22:00:00
"Staunton-Waynesboro, VA",44420,277,44420,310M400US44420,M1,2595073753,10049978,125774.0,"POLYGON ((-79.43810 38.21623, -79.30869 38.382...",0.0,2023-02-03 22:00:00


In [25]:
df_geo.to_pickle("../data/google_trend/DerailmentHourlyGeo.pkl", compression="gzip")

In [26]:
geometry = [Point(xy) for xy in zip(df_daily_search["Longitude"], df_daily_search["Latitude"])]
df_geo = gpd.GeoDataFrame(df_daily_search, geometry=geometry, crs="EPSG:4269")
df_geo = gpd.sjoin(df_geo, df_shp, predicate="within", how="left")

dfs = list()
df_geo["Date"] = pd.to_datetime(df_geo["EST"])
df_geo = df_geo.set_index(["Date"])
for date in df_geo.index.drop_duplicates():
    df = df_geo.loc[date]
    gdf = df_shp.set_index(["NAME"])
    gdf["Count"] = df.groupby(["NAME"]).sum()["DMA_Search"]
    gdf["Count"] = gdf["Count"].fillna(0)
    gdf["Date"] = date.strftime("%Y%m%d")
    dfs.append(gdf)
df_geo = pd.concat(dfs).sort_values(["Date"])
df_geo.head(5)

,GEOID,CSAFP,CBSAFP,AFFGEOID,LSAD,ALAND,AWATER,Population,geometry,Count,Date
NAME,,,,,,,,,,,
"Muskogee, OK",34780,538,34780,310M400US34780,M2,2098872174,76147852,66146.0,"POLYGON ((-95.76617 35.85640, -95.65042 35.856...",0.0,20230203
"Greenville, MS",24740,NaN,24740,310M400US24740,M2,1877059426,94506019,43687.0,"POLYGON ((-91.21567 33.52942, -90.76351 33.529...",0.0,20230203
"Columbia, SC",17900,192,17900,310M400US17900,M1,9591002754,339370306,838250.0,"POLYGON ((-81.99071 33.99661, -81.87008 34.135...",0.0,20230203
"Twin Falls, ID",46300,NaN,46300,310M400US46300,M1,6524657692,31157724,116905.0,"POLYGON ((-115.03770 42.76841, -115.03777 42.9...",77.0,20230203
"Staunton-Waynesboro, VA",44420,277,44420,310M400US44420,M1,2595073753,10049978,125774.0,"POLYGON ((-79.43810 38.21623, -79.30869 38.382...",0.0,20230203


In [ ]:
df_geo.to_pickle("../data/google_trend/DerailmentDailyGeo.pkl", compression="gzip")